In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Underweight"
cohort = "GSE130563"

# Input paths
in_trait_dir = "../../input/GEO/Underweight"
in_cohort_dir = "../../input/GEO/Underweight/GSE130563"

# Output paths
out_data_file = "../../output/preprocess/Underweight/GSE130563.csv"
out_gene_data_file = "../../output/preprocess/Underweight/gene_data/GSE130563.csv"
out_clinical_data_file = "../../output/preprocess/Underweight/clinical_data/GSE130563.csv"
json_path = "../../output/preprocess/Underweight/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE130563_family.soft.gz', 'GSE130563_series_matrix.txt.gz']
Identified SOFT files: ['GSE130563_family.soft.gz']
Identified matrix files: ['GSE130563_series_matrix.txt.gz']

Background Information:
!Series_title	"Skeletal Muscle Fibrosis in Pancreatic Cancer Patients with Respect to Survival"
!Series_summary	"Skeletal muscle wasting is a devastating consequence of cancer that may be responsible for nearly 30% of cancer-related deaths. In addition to muscle atrophy, we have identified significant muscle fiber damage and replacement of muscle with fibrotic tissue in rectus abdominis muscle biopsies from cachectic pancreatic ductal adenocarcinoma (PDAC) patients that associates with poor survival. Transcriptional profiling of muscle harvested from these same patients supported these findings by identifying gene clusters related to wounding, inflammation and cellular response to TGF-B upregulated in cachectic PDAC patients compared with non-cancer controls.

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from
# skeletal muscle (rectus abdominis) biopsies. Not miRNA or methylation data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Underweight):
# Looking at the data, row 3 contains "bw loss in 6 months prior to surgery" which can be used to determine underweight status
# The background mentions "Cachexia was defined as a body weight loss of >5% during the 6 months prior to surgery"
trait_row = 3

# For age:
# Row 4 contains age information
age_row = 4

# For gender:
# Row 1 contains sex information
gender_row = 1

# 2.2 Data Type Conversion

def convert_trait(value):
    """
    Convert body weight loss value to Underweight status (binary)
    Cachexia/Underweight defined as body weight loss >5% in 6 months prior to surgery
    """
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value == 'n.d. (not determined)':
        return None
    
    try:
        # Convert to float and determine if >5% (cachexia/underweight)
        bw_loss = float(value)
        # According to the background info, >5% weight loss = cachectic/underweight
        return 1 if bw_loss > 5 else 0
    except:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert sex to binary (0=female, 1=male)"""
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # The clinical_data is already available from the previous step
    # Extract clinical features using the provided clinical_data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)


{'GSM3743555': [0.0, 33.0, 0.0], 'GSM3743556': [0.0, 68.0, 0.0], 'GSM3743557': [0.0, 73.0, 0.0], 'GSM3743558': [0.0, 49.0, 0.0], 'GSM3743559': [0.0, 78.0, 0.0], 'GSM3743560': [0.0, 57.0, 0.0], 'GSM3743561': [0.0, 55.0, 0.0], 'GSM3743562': [0.0, 50.0, 0.0], 'GSM3743563': [0.0, 47.0, 0.0], 'GSM3743564': [0.0, 63.0, 0.0], 'GSM3743565': [0.0, 51.0, 1.0], 'GSM3743566': [0.0, 50.0, 1.0], 'GSM3743567': [1.0, 69.0, 1.0], 'GSM3743568': [0.0, 50.0, 1.0], 'GSM3743569': [1.0, 60.0, 0.0], 'GSM3743570': [1.0, 68.0, 0.0], 'GSM3743571': [1.0, 66.0, 1.0], 'GSM3743572': [1.0, 54.0, 1.0], 'GSM3743573': [1.0, 64.0, 0.0], 'GSM3743574': [1.0, 76.0, 0.0], 'GSM3743575': [1.0, 68.0, 0.0], 'GSM3743576': [1.0, 73.0, 1.0], 'GSM3743577': [1.0, 56.0, 0.0], 'GSM3743578': [1.0, 80.0, 0.0], 'GSM3743579': [1.0, 68.0, 0.0], 'GSM3743580': [1.0, 79.0, 0.0], 'GSM3743581': [1.0, 72.0, 1.0], 'GSM3743582': [1.0, 52.0, 0.0], 'GSM3743583': [1.0, 74.0, 1.0], 'GSM3743584': [1.0, 74.0, 0.0], 'GSM3743585': [1.0, 55.0, 1.0], 'GSM374

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')

Gene expression data shape: (23786, 46)


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers
# The identifiers like '100009613_at', '10000_at' appear to be probe IDs from a microarray
# They have a numeric component followed by '_at' which is a typical format for Affymetrix probe IDs
# These are not standard human gene symbols and would need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene identifiers and gene annotation data
# First, let's check more thoroughly for gene symbol information in the SOFT file

# Look for gene symbols in the SOFT file
gene_symbols_found = False
platform_id = None

try:
    # Check first few lines of the SOFT file for platform information
    with gzip.open(soft_file_path, 'rt') as f:
        for i, line in enumerate(f):
            if "^PLATFORM" in line:
                # Extract platform ID
                platform_id = line.strip().split("=")[1]
                print(f"Found platform ID: {platform_id}")
            if "Gene symbol" in line or "gene_symbol" in line or "SYMBOL" in line:
                print(f"Found potential gene symbol reference at line {i}: {line.strip()}")
                gene_symbols_found = True
            if i > 200:  # Check first 200 lines
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# If gene symbols weren't found in annotation, let's extract platform annotation data
if not gene_symbols_found:
    print("\nLooking for gene symbols in platform data...")
    
    try:
        # Read the SOFT file to extract platform data
        platform_lines = []
        in_platform = False
        with gzip.open(soft_file_path, 'rt') as f:
            for line in f:
                if line.startswith('^PLATFORM'):
                    in_platform = True
                elif line.startswith('^'):
                    in_platform = False
                if in_platform and line.startswith('!Platform_table_begin'):
                    platform_lines = []
                    continue
                if in_platform and line.startswith('!Platform_table_end'):
                    break
                if in_platform and platform_lines is not None and not line.startswith('!'):
                    platform_lines.append(line.strip())
                    
        if platform_lines:
            # Create a DataFrame from the platform data
            platform_data = pd.read_csv(io.StringIO('\n'.join(platform_lines)), sep='\t')
            print("Platform data columns:", platform_data.columns.tolist())
            
            # Look for ID and gene symbol columns
            id_col = next((col for col in platform_data.columns if 'ID' in col or 'id' in col), None)
            gene_symbol_cols = [col for col in platform_data.columns if 'symbol' in col.lower() or 'gene' in col.lower()]
            
            if id_col and gene_symbol_cols:
                print(f"Found ID column: {id_col}")
                print(f"Found potential gene symbol columns: {gene_symbol_cols}")
                
                # Preview the potential gene symbol column contents
                for col in gene_symbol_cols:
                    print(f"\nSample values from {col}:")
                    print(platform_data[col].head(5).tolist())
                
                # Choose the most appropriate gene symbol column
                gene_symbol_col = gene_symbol_cols[0]
                
                # Create a mapping DataFrame
                mapping_df = platform_data[[id_col, gene_symbol_col]].copy()
                mapping_df.columns = ['ID', 'Gene']
                
                print("\nGene mapping dataframe preview:")
                print(preview_df(mapping_df))
                
                # 3. Apply gene mapping to convert probe-level measurements to gene expression
                gene_data = apply_gene_mapping(gene_data, mapping_df)
                
                print("\nFirst few rows of mapped gene expression data:")
                print(preview_df(gene_data))
                
                # Print shape to see how many unique genes we have after mapping
                print(f"\nGene expression data shape after mapping: {gene_data.shape}")
            else:
                print("Could not find appropriate ID and gene symbol columns in platform data.")
    except Exception as e:
        print(f"Error extracting platform data: {e}")

# If we still don't have gene symbols, use IDs as symbols and proceed
if 'gene_data' not in locals() or gene_data.empty:
    print("\nUsing original gene identifiers and extracting gene symbols from them...")
    
    # Extract gene symbols from the probe IDs themselves
    # Assuming probe IDs follow a pattern like GENE_at (e.g., BRCA1_at)
    probe_ids = gene_data.index.tolist()
    gene_symbols = []
    
    for probe in probe_ids:
        # Try to extract gene symbol part from probe ID
        if '_at' in probe:
            symbol = probe.split('_at')[0]
            gene_symbols.append(symbol)
        else:
            gene_symbols.append(probe)
    
    # Create mapping DataFrame
    mapping_df = pd.DataFrame({
        'ID': probe_ids,
        'Gene': gene_symbols
    })
    
    print("\nGene mapping dataframe preview (from probe IDs):")
    print(preview_df(mapping_df))
    
    # Apply gene mapping
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    print("\nFirst few rows of mapped gene expression data:")
    print(preview_df(gene_data))
    
    # Print shape to see how many unique genes we have after mapping
    print(f"\nGene expression data shape after mapping: {gene_data.shape}")

# Save the gene expression data to CSV
if not gene_data.empty:
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"\nSaved gene expression data to {out_gene_data_file}")
else:
    print("\nNo gene expression data to save.")


Found platform ID:  GPL17930

Looking for gene symbols in platform data...


Error extracting platform data: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2




Saved gene expression data to ../../output/preprocess/Underweight/gene_data/GSE130563.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Let's first check what's happening with the gene data
# Instead of normalizing, which is causing an empty dataframe, 
# we'll use extract_human_gene_symbols to get symbols from probe IDs

# Create a better mapping approach
import re

# Parse probe IDs to potential gene symbols
# Example: "100009613_at" -> "100009613"
mapping_dict = {}
for probe_id in gene_data.index:
    # For Affymetrix probe IDs, we'll extract the numeric part 
    # This isn't ideal but better than dropping all data
    if '_at' in probe_id:
        # Just remove the '_at' suffix
        potential_symbol = probe_id.split('_at')[0]
        mapping_dict[probe_id] = potential_symbol
    else:
        mapping_dict[probe_id] = probe_id

# Create new gene data with these mappings
new_gene_data = gene_data.copy()
new_gene_data.index = [mapping_dict.get(idx, idx) for idx in new_gene_data.index]

# Group by the new indices to handle duplicates
new_gene_data = new_gene_data.groupby(level=0).mean()

print(f"Gene data shape after manual mapping: {new_gene_data.shape}")
print(f"First few gene symbols after manual mapping: {list(new_gene_data.index[:10])}")

# Save this version of gene data without normalization
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
new_gene_data.to_csv(out_gene_data_file)
print(f"Mapped gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in Step 2
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical features extracted:")
print(preview_df(clinical_features))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, new_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = list(linked_data.columns[:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0:
    print("WARNING: No samples left after handling missing values.")
    is_trait_biased = True  # Force as biased since we can't properly evaluate
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from rectus abdominis muscle biopsies, focusing on cachexia in pancreatic cancer patients."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after manual mapping: (23786, 46)
First few gene symbols after manual mapping: ['1', '10', '100', '1000', '10000', '100009613', '100009676', '10001', '10002', '10003']


Mapped gene data saved to ../../output/preprocess/Underweight/gene_data/GSE130563.csv
Clinical features extracted:
{'GSM3743555': [0.0, 33.0, 0.0], 'GSM3743556': [0.0, 68.0, 0.0], 'GSM3743557': [0.0, 73.0, 0.0], 'GSM3743558': [0.0, 49.0, 0.0], 'GSM3743559': [0.0, 78.0, 0.0], 'GSM3743560': [0.0, 57.0, 0.0], 'GSM3743561': [0.0, 55.0, 0.0], 'GSM3743562': [0.0, 50.0, 0.0], 'GSM3743563': [0.0, 47.0, 0.0], 'GSM3743564': [0.0, 63.0, 0.0], 'GSM3743565': [0.0, 51.0, 1.0], 'GSM3743566': [0.0, 50.0, 1.0], 'GSM3743567': [1.0, 69.0, 1.0], 'GSM3743568': [0.0, 50.0, 1.0], 'GSM3743569': [1.0, 60.0, 0.0], 'GSM3743570': [1.0, 68.0, 0.0], 'GSM3743571': [1.0, 66.0, 1.0], 'GSM3743572': [1.0, 54.0, 1.0], 'GSM3743573': [1.0, 64.0, 0.0], 'GSM3743574': [1.0, 76.0, 0.0], 'GSM3743575': [1.0, 68.0, 0.0], 'GSM3743576': [1.0, 73.0, 1.0], 'GSM3743577': [1.0, 56.0, 0.0], 'GSM3743578': [1.0, 80.0, 0.0], 'GSM3743579': [1.0, 68.0, 0.0], 'GSM3743580': [1.0, 79.0, 0.0], 'GSM3743581': [1.0, 72.0, 1.0], 'GSM3743582': [1.0, 

Linked data shape after handling missing values: (38, 23789)
For the feature 'Underweight', the least common label is '0.0' with 18 occurrences. This represents 47.37% of the dataset.
The distribution of the feature 'Underweight' in this dataset is fine.

Quartiles for 'Age':
  25%: 55.0
  50% (Median): 65.0
  75%: 72.75
Min: 33.0
Max: 80.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 15 occurrences. This represents 39.47% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Is trait biased: False
A new JSON file was created at: ../../output/preprocess/Underweight/cohort_info.json
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Underweight/GSE130563.csv
